In [10]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from spint.gravity import Gravity, Production
from spopt import MaxPHeuristic, RegionKMeansHeuristic, WardSpatial

In [4]:
county_level = pd.read_csv('../data/weekly_county2county_08_10.csv', usecols=['geoid_o', 'geoid_d', 'visitor_flows', 'pop_flows'])

In [8]:
ct_level = pd.read_csv('../data/weekly_ct2ct_08_10.csv', nrows=10, usecols=['geoid_o', 'geoid_d', 'visitor_flows', 'pop_flows'])

In [9]:
ct_level

,geoid_o,geoid_d,visitor_flows,pop_flows
0,1001020100,1001020100,8,85.0
1,1001020100,1001020200,109,1171.0
2,1001020100,1001020300,105,1128.0
3,1001020100,1001020400,29,311.0
4,1001020100,1001020500,328,3525.0
5,1001020100,1001020600,20,214.0
6,1001020100,1001020700,109,1171.0
7,1001020100,1001020801,13,139.0
8,1001020100,1001020802,12,128.0
9,1001020100,1001020900,4,42.0
